# Atividade Final: Machine Learning

O Grande Rei de Lumi convocou uma reunião do Conselho Real a fim de discutir o futuro da saúde pública de seu reino. Diana Didatolov, que estava presente, mostrou os dados coletados pela Organização Mística da Saúde (OMS) e arquivados na Biblioteca Real, os quais continham informações preciosas a respeito da saúde pública das várias regiões de Lumi nos últimos anos. O rei, que se preocupava muito com a qualidade de vida de seus anciãos, desejava prever a futura expectativa de vida das diversas regiões do reino. O que Diana não sabia, no entanto, era como obter tal informação a partir de uma coleção de dados tão extensa.

Diante desse problema, o Grande Rei convocou a guilda mais poderosa de Lumi, composta por dois bravos cavaleiros e duas magas poderosas: Os Guardiões de Ba'IA. 

Os Cavaleiros Maria e Izaque, que possuem muitos contatos através do reino, decidiram perguntar aos sábios do Clã Vizin'Hos qual seria a solução do problema.

A Maga Dara, por outro lado, achou mais sábio perguntar à grande Árvore do Conhecimento a resposta.

Já a Feiticeira Alice, não satisfeita, viajou até a Floresta da Elucidação em busca de uma maneira de ajudar o Rei. 

Alice, Izaque, Maria e Dara retornaram com respostas diferentes a respeito da expectativa de vida de cada região. Mas como saber qual desses métodos se aproximou mais da realidade? 

Descrição do Projeto:
Comparar diversos modelos de previsão (KNN-vizinhos, Árvore de Decisão e Floresta de Decisão) e métodos de tratamento de dados quanto a sua eficiência na predição. 

Iniciamos importando todas as bibliotecas e métodos que serão utilizadas para a atividade. 

In [1]:
import pandas as pd
import numpy as np
import statistics as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Tratamento de Dados

O primeiro passo para a realização da atividade é o tratamento dos dados do nosso DataFrame. Ao longo desse tratamento, criaremos diferentes versões do data frame e, ao final, usaremos cada um deles para treinar diferentes modelos. Por fim, iremos avaliar e comparar a eficiência de cada um desses modelos na predição. <br>

# Primeiros Passos

Primeiro vamos salvar nosso dataframe numa variável, assim como obter algumas informações básicas sobre ele, como número de linhas e colunas. 

In [2]:
raw_df = pd.read_csv('life_expect.csv') #salvar o dataframe na variável 'raw_df'
print(raw_df) #mostrar o dataframe
print(raw_df.columns) #mostrar as colunas do dataframe

          Country  Year      Status  Life expectancy   Adult Mortality  \
0     Afghanistan  2015  Developing              65.0            263.0   
1     Afghanistan  2014  Developing              59.9            271.0   
2     Afghanistan  2013  Developing              59.9            268.0   
3     Afghanistan  2012  Developing              59.5            272.0   
4     Afghanistan  2011  Developing              59.2            275.0   
...           ...   ...         ...               ...              ...   
2933     Zimbabwe  2004  Developing              44.3            723.0   
2934     Zimbabwe  2003  Developing              44.5            715.0   
2935     Zimbabwe  2002  Developing              44.8             73.0   
2936     Zimbabwe  2001  Developing              45.3            686.0   
2937     Zimbabwe  2000  Developing              46.0            665.0   

      infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   \
0                62     0.01    

In [3]:
print('O dataset possui', raw_df.shape[0], 'instâncias e', raw_df.shape[1], 'atributos.')

O dataset possui 2938 instâncias e 22 atributos.


Nosso dataset possui dados em diferentes unidades: alguns estão em anos, porcentagem, unidade por milhar, etc. Para nos guiar a respeito da unidade utilizada em cada coluna, criei um dicionário abaixo.

In [4]:
#Explicação de cada coluna segundo o Kaggle
info_colunas = {
    'Country' : {'Country'},
    'Year' : {'Year of the data'},
    'Status' : {'developing or developed country?'},
    'Life expectancy ' : {'Life Expectancy in age'},
    'Adult Mortality' : {'Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population) '},
    'infant deaths' : {'Number of Infant Deaths per 1000 population'},
    'Alcohol' : {'Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)'},
    'percentage expenditure' : {'Expenditure on health as a percentage of Gross Domestic Product per capita(%) '},
    'Hepatitis B' : {'Hepatitis B (HepB) immunization coverage among 1-year-olds (%)'},
    'Measles ' : {'Measles - number of reported cases per 1000 population '},
    ' BMI ' : {'Average Body Mass Index of entire population '},
    'under-five deaths ' : {'Number of under-five deaths per 1000 population'},
    'Polio' : {'Polio (Pol3) immunization coverage among 1-year-olds (%) '},
    'Total expenditure' : {'General government expenditure on health as a percentage of total government expenditure (%)'},
    'Diphtheria ' : {'Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%) '},
    ' HIV/AIDS' : {'Deaths per 1 000 live births HIV/AIDS (0-4 years) '},
    'GDP' : {'Gross Domestic Product per capita (in USD)'},
    'Population' : {'Population of the country '},
    ' thinness  1-19 years' : {'Prevalence of thinness among children and adolescents for Age 10 to 19 (% ) '},
    ' thinness 5-9 years' : {'Prevalence of thinness among children for Age 5 to 9(%) '},
    'Income composition of resources' : {'Human Development Index in terms of income composition of resources (index ranging from 0 to 1) '},
    'Schooling' : {'Number of years of Schooling(years) '}
    }

Decidimos que os dados contidos nas colunas "Country" e "Year" não serão úteis para a nossa análise, de forma que eles serão removidos do dataset. Verificamos, também, que há uma coluna contendendo dados em string a respeito do status de desenvolvimento de cada país (em que as opções possíveis são 'desenvolvido' e 'em desenvolvimento'). Aplicaremos um método de mapeamento para transformar esses valores em string em valores booleanos 0 e 1. 

In [5]:
raw_1_df = raw_df.drop( #armazenamos na variável 'raw_1_df' o dataframe sem as colunas 'country' e 'year'
    ["Country", 'Year'], axis=1)
#print(raw_1_df)

In [6]:
#define um dicionário que determina pelo que cada valor deve ser substituido na série
mapear_status = {'Developing' : 1, 'Developed' : 0} 

#cria uma copia do df onde será realizada a iteração
raw_copy_df = raw_1_df.copy()
#print(raw_copy_df.copy)

#Realiza o mapeamento
raw_1_df['Status'] = raw_copy_df['Status'].map(mapear_status, na_action=None)

print(raw_1_df)

      Status  Life expectancy   Adult Mortality  infant deaths  Alcohol  \
0          1              65.0            263.0             62     0.01   
1          1              59.9            271.0             64     0.01   
2          1              59.9            268.0             66     0.01   
3          1              59.5            272.0             69     0.01   
4          1              59.2            275.0             71     0.01   
...      ...               ...              ...            ...      ...   
2933       1              44.3            723.0             27     4.36   
2934       1              44.5            715.0             26     4.06   
2935       1              44.8             73.0             25     4.43   
2936       1              45.3            686.0             25     1.72   
2937       1              46.0            665.0             24     1.68   

      percentage expenditure  Hepatitis B  Measles    BMI   \
0                  71.279624         

Pronto! Agora que nosso dataframe possui apenas as colunas que utilizaremos para a nossa predição e todos os dados estão em formato número, podemos prosseguir com o tratamento dos dados. 

# Dados NaN

No tratamendo de Dados, existem diferentes formas de se lidar com dados NaN. Aqui, iremos comparar 3 métodos diferentes: <br>

1. Preencher os dados NaN com a mediana <br>
2. Excluir todas as linhas contendo pelo menos um dado NaN <br>
3. Excluir colunas com grande volume de dados NaN e só então excluir as linhas contendo pelo menos um dado NaN <br>
<br>
Entendemos que essas não são as únicas formas possíveis de se tratar dados NaN, sendo possível até mesmo unir as técnicas acima. No entanto, elencamos 3 opções a fim de compará-las e determinar a melhor estratégia para o nosso conjunto de dados. 


Primeiro, vamos iniciar utilizando o método `.info()` para saber se o nosso dataset possui dados NaN e se há colunas específicas com um maior número de dados NaN.

In [7]:
raw_1_df.info() #Obtém informações quanto ao número de valores não nulos no dataframe original

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Status                           2938 non-null   int64  
 1   Life expectancy                  2928 non-null   float64
 2   Adult Mortality                  2928 non-null   float64
 3   infant deaths                    2938 non-null   int64  
 4   Alcohol                          2744 non-null   float64
 5   percentage expenditure           2938 non-null   float64
 6   Hepatitis B                      2385 non-null   float64
 7   Measles                          2938 non-null   int64  
 8    BMI                             2904 non-null   float64
 9   under-five deaths                2938 non-null   int64  
 10  Polio                            2919 non-null   float64
 11  Total expenditure                2712 non-null   float64
 12  Diphtheria          

Observamos que as colunas 'Alcohol', 'Life expectancy', 'Adult Mortality', 'Hepatitis B', 'BMI', 'Polio', 'Total expenditure', 'Diphtheria', 'GDP', 'Population', 'thinness  1-19 years','thinness 5-9 years','Income composition of resources' e 'Schooling' possuem dados NaN, sendo as colunas `Population`, `GDP` e `Hepatites B` aquelas com o maior número de dados NaN. <br>
A partir do dataset original, vamos criar 3 novos datasets, cada um utilizando um dos métodos de se lidar com dados NaN mencionados acima. Os datasets serão nomeados a partir das letras gregas alpha, beta e gamma e serão futuramente comparados a fim de verificar qual leva a uma melhor acurácia na predição.  

## Método 1: Remover todas as linhas contendo dados NaN (Alpha)

In [8]:
alpha_df = raw_1_df.dropna() #armazenamos na variável 'alpha_df' o dataframe sem as linhas com valores NaN
#print(alpha_df)
print('Após remover as linhas com dados faltantes, obtivemos um dataset com', 
      alpha_df.shape[0], 'instâncias e', alpha_df.shape[1], 'atributos.')

Após remover as linhas com dados faltantes, obtivemos um dataset com 1649 instâncias e 20 atributos.


## Método 2: Preencher linhas contendo dados NaN utilizando a mediana (Beta)

Para preencher as linhas contendo dados NaN utilizaremos o método `Simple Imputer` do Sklearn e a estratégia de preenchimento `mediana`. Como temos uma coluna de dados booleanos 0 e 1 ('Status'), não podemos utilizar essa estratégia sobre esses dados, afinal seria imputado um valor entre 0 e 1! Nesse caso, teríamos que aplicar a estratégia `mode` nessa coluna. No entanto, nossa coluna 'Status' não possui nenhum dado NaN, de forma que não seja necessário usar nenhum método de imputamento sobre ela. 

In [9]:
beta_df = raw_1_df #criamos uma cópia do dataset original

#Definir em quais colunas desejamos preencher os dados NaN faltantes
colunas = ['Adult Mortality', 'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling']

#Define o método de imputamento e a estratégia
imputer = SimpleImputer(missing_values = np.nan, 
                        strategy ='mean')

#Fita os dados sobre as colunas que desejamos imputar
imputer = imputer.fit(raw_1_df[colunas])

#Cria um novo dataset com as colunas preenchidas pela mediana!
beta_df[colunas] = imputer.transform(raw_1_df[colunas])

In [21]:
#print(beta_df)
#beta_df.info()

## Método 3: Excluir colunas com grande volume de dados NaN (Gamma)

Por fim, iremos criar um terceiro dataset a partir de uma técnica onde identificamos colunas com um grande número de valores NaN e as removemos. Só então removemos as linhas contendo pelo menos um dado NaN. Para isso, iremos remover as colunas `Hepatitis B, Population e GDP`. 

In [22]:
#armazenamos na variável 'df_semcolunas_nan' o dataframe sem as colunas com muitos valores nan e as colunas país e ano
df_semcolunas_nan = raw_1_df.drop(
    ['Hepatitis B', 'Population', 'GDP'], axis=1)
#print(df_semcolunas_nan)


#armazenamos na variável 'final_df_semcolunas_nan' o dataframe sem as linhas com valores NaN
gamma_df = df_semcolunas_nan.dropna()
print('Após remover as linhas com dados faltantes, obtivemos um dataset com', 
      gamma_df.shape[0], 'instâncias e', gamma_df.shape[1], 'atributos.')

Após remover as linhas com dados faltantes, obtivemos um dataset com 2928 instâncias e 17 atributos.


Mas e se essas colunas que removemos forem essenciais para a predição do nosso target? Para verificar se esse é o caso, podemos utilizar o método SHAP, que verifica o grau de importância de cada atributo na predição. 

## SHAP

Aplicar o SHAP, plotar a tabelinha. Tudo indica que vai mostrar que esses três atributos não influenciam tanto no target, já que quando fizemos isso em Dados 2 teve uma melhora na performance sem essas colunas (então elas provavelmente não eram tão importantes assim). É só fazer o gráfico e colocar um parágrafo falando "com isso sabemos que essas colunas não são tão essenciais para a predição, de forma que podemos removê-las a fim de verificar se há uma melhora na performance" ou algo desse tipo. 

Pronto, agora temos 3 datasets criados a partir de diferentes métodos de tratamento de dados NaN! Cada um desses Datasets possui 0 valores NaN, deixando-os prontos para a próxima fase do tratamento de dados. 

In [12]:
#print(alpha_df)
#print(beta_df)
#print(gamma_df)
#alpha_df.info()
#beta_df.info()
#gamma_df.info()

# Normalização

Utilizaremos 3 modelos de Machine Learning com os nossos dados: KNN-Vizinhos, Árvore de Decisão e Floresta de Decisão. Modelos KNN realizam sua previsão com base na distância entre os dados, de forma que colunas contendo dados com dimensões muito maiores que as outras podem apresentar um peso maior na decisão que não se reflete na realidade do problema. Por isso, é recomendável realizar uma normalização dos dados antes de prosseguir. <br>

Para verificar a influência da normalização (ou a falta dela) nos dados, iremos comparar também a eficiência dos nossos modelos. Espera-se que seja percebida alguma diferença na predição com o modelo KNN e nenhuma (ou pouca, dada a aleatoriedade inerente a esses modelos) diferença para os modelos árvore e floresta de decisão. <br>

Optamos por comparar criar duas versões para cada um dos datasets já criados anteriormente: <br>

a. Versão sem normalizar <br>
b. Versão utilizando Normalização Padrão <br> 

Assim teremos 6 datasets finais:

1. alpha_df
2. alpha_norm_df
3. beta_df
4. beta_norm_df
5. gamma_df
6. gamma_norm_df


In [23]:
#Lista com as colunas onde será realizada a normalização.
Colunas = ['Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling']

#Como o dataframe gamma possui menos colunas que as outras, criamos uma lista separada
Colunas_gamma = ['Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling']

#Cria cópias de cada um dos 3 datasets
norm_alpha_df = alpha_df
norm_beta_df = beta_df
norm_gamma_df = gamma_df

#Define o método de normalização que será utilizado
normalizador = StandardScaler()

#Fita os dados das colunas desejadas 
#Realiza a normalização sobre os dados Alpha
normalizador.fit(alpha_df[Colunas])
norm_alpha_df[Colunas] = normalizador.transform(alpha_df[Colunas])

#Fita os dados das colunas desejadas 
#Realiza a normalização sobre os dados Beta
normalizador.fit(beta_df[Colunas])
norm_beta_df[Colunas] = normalizador.transform(beta_df[Colunas])

#Fita os dados das colunas desejadas 
#Realiza a normalização sobre os dados Gamma
normalizador.fit(gamma_df[Colunas_gamma])
norm_gamma_df[Colunas_gamma] = normalizador.transform(gamma_df[Colunas_gamma])

C:\Users\iasodara23005\AppData\Local\Temp\ipykernel_776\1795983486.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norm_alpha_df[Colunas] = normalizador.transform(alpha_df[Colunas])
C:\Users\iasodara23005\AppData\Local\Temp\ipykernel_776\1795983486.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  norm_gamma_df[Colunas_gamma] = normalizador.transform(gamma_df[Colunas_gamma])


Pronto, agora nossos dados estão todos normalizados, tratados e prontos para seguir para o próximo passo!

# Referências

1. Como preencher dados NaN com o SimpleImputer: https://medium.com/data-hackers/tratamento-e-transforma%C3%A7%C3%A3o-de-dados-nan-uma-vis%C3%A3o-geral-e-pr%C3%A1tica-54efa9fc7a98